In [ ]:
import os
import utils

import torch
import torch.optim as optim

from rnn import RNN, masked_mse_loss, train_rnn

In [ ]:
NUM_SEQUENCES = 121
# (Delta_t, P_t, x_t)
BATCH_SIZE = 16
HIDDEN_SIZE = 256
NUM_LAYERS = 3
LEARNING_RATE = 1e-3
NUM_EPOCH_CONVERGENCE = 5

In [ ]:
train_dataset, valid_dataset, test_dataset = utils.train_valid_test_split(
    range(1, NUM_SEQUENCES + 1), recurrent=True, train_ratio=0.7)

train_inputs, train_targets, train_lengths = train_dataset
valid_inputs, valid_targets, valid_lengths = valid_dataset
test_inputs, test_targets, test_lengths = test_dataset

# Standardization statistics
train_max_length = train_lengths.max().item()
num_train = train_inputs.size(1)
input_size = train_inputs.size(-1)
output_size = train_targets.size(-1)
timesteps = torch.arange(train_max_length).expand(num_train, train_max_length)
masks = (timesteps < train_lengths.unsqueeze(1)).T.unsqueeze(-1)

train_inputs_flatten = train_inputs[masks.expand(-1, -1, input_size)]
train_inputs_flatten = train_inputs_flatten.view(-1, input_size)
train_targets_flatten = train_targets[masks.expand(-1, -1, output_size)]
train_targets_flatten = train_targets_flatten.view(-1, output_size)

mean_inputs = train_inputs_flatten.mean(dim=0)
std_inputs = train_inputs_flatten.std(dim=0)
mean_targets = train_targets_flatten.mean(dim=0)
std_targets = train_targets_flatten.std(dim=0)

rnn = RNN(
    cell='gru',
    input_size=train_inputs.size(2),
    hidden_size=HIDDEN_SIZE,
    output_size=train_targets.size(2),
    num_layers=NUM_LAYERS,
    mean_inputs=mean_inputs,
    std_inputs=std_inputs,
    mean_targets=mean_targets,
    std_targets=std_targets)

opt = optim.Adam(rnn.parameters(), lr=LEARNING_RATE)

train_rnn(rnn, opt, train_inputs, train_targets, train_lengths,
    valid_inputs, valid_targets, valid_lengths, BATCH_SIZE,
    NUM_EPOCH_CONVERGENCE)

In [ ]:
with torch.no_grad():
    test_preds, _ = rnn(test_inputs)

utils.show_sample_sequence(
    test_targets, test_preds, test_lengths, recurrent=True)

mse_loss = masked_mse_loss(test_preds, test_targets, test_lengths,
                           test_lengths.max().item())
print('Test MSE Loss: {:,.4f}'.format(mse_loss.item()))

os.makedirs('../weights/', exist_ok=True)
weights_file = '../weights/rnn-L{}H{}.pth'.format(NUM_LAYERS, HIDDEN_SIZE)
torch.save(rnn.state_dict(), weights_file)

Caution: The RNN is trained to predict only one step every 20 time steps
Reminder: The MLP without data standardization was 90 on the test set
Reminder: The MLP with data standardization was 30 on the test set

In [ ]:
utils.show_sample_sequence(
    test_targets, test_preds, test_lengths, recurrent=True)